Load necessary packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
import csv
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import metrics
import pandas as pd
import tensorflow as tf
import os

from keras.preprocessing import image
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization, Conv2D, Input, Dense, Dropout, MaxPool2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras import callbacks
from keras.callbacks import EarlyStopping
from keras.applications.vgg16 import VGG16
from keras.layers.convolutional import ZeroPadding2D
from keras.optimizers import Adamax
from keras.applications import ResNet50

result = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

Using TensorFlow backend.


Download Training Data from Google Drive

In [2]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '1wn1WdiMqfaBkHjLGgAGo7mgB2iCCpabk'})  #From shareable link
download.GetContentFile('fer2013.csv')
!ls

adc.json  fer2013.csv  sample_data


Load data-set and splite into training-validation and test dataset

In [3]:
def load_data():
	train_data_X, train_data_Y = [], []
	validation_data_X, validation_data_Y = [], []
	test_data_X, test_data_Y = [], []
	with open("fer2013.csv", 'r') as csvfile:
		datareader = csv.reader(csvfile, delimiter =',')
		headers = next(datareader)
		print(headers) 
		for row in datareader:
			emotion = int(row[0])
			usage = row[2]
			pixels = [int(p) for p in row[1].split()]
			if usage == "Training":
				train_data_X.append(pixels)
				train_data_Y.append(emotion)
			elif usage == "PrivateTest":
				validation_data_X.append(pixels)
				validation_data_Y.append(emotion)
			else:
				test_data_X.append(pixels)
				test_data_Y.append(emotion)
	train_data_X, train_data_Y = np.array(train_data_X)/255.0, np.array(train_data_Y)
	validation_data_X, validation_data_Y = np.array(validation_data_X)/255.0, np.array(validation_data_Y)
	test_data_X, test_data_Y = np.array(test_data_X)/255.0, np.array(test_data_Y)

  
	print("train_data ", train_data_X.shape)
	print("validation_data ", validation_data_X.shape)
	print("test_data ", test_data_X.shape)
	return train_data_X, train_data_Y, validation_data_X, validation_data_Y, test_data_X, test_data_Y

train_data_X, train_data_Y, validation_data_X, validation_data_Y, test_data_X, test_data_Y = load_data()

['emotion', 'pixels', 'Usage']
train_data  (28709, 2304)
validation_data  (3589, 2304)
test_data  (3589, 2304)


Resnet50

In [0]:
# Reshape array
train_data_X = train_data_X.reshape(28709, 48, 48)
validation_data_X = validation_data_X.reshape(3589, 48, 48)
test_data_X = test_data_X.reshape(3589, 48, 48)

train_data_Y = to_categorical(train_data_Y, num_classes=7)
validation_data_Y = to_categorical(validation_data_Y, num_classes=7)
test_data_Y = to_categorical(test_data_Y, num_classes=7)


IMAGE_SHAPE = (48, 48, 1)
raw_data_csv_file_name = 'fer2013.csv'
N_TEST = 25000

# 7 Classes
num_classes = 7
resnetPreTrained = ResNet50(include_top=False, weights='imagenet', input_shape=(200,200,3))


x_train_feature_map = np.empty([N_TEST, 2048])

for i in range(5):
    print("loading images {}".format(i*5000))
    load_range = (i*5000)
    f_l = np.empty([int(N_TEST/20), 200, 200, 3])
    for index, item in enumerate(f_l[load_range:load_range+4999]):  # Refill the list
        for index, item in enumerate(f_l):  # Refill the list
            for d in range(3):
                item[0:48, 0:48, d] = train_data_X[index]


    picture_train_features = resnetPreTrained.predict(f_l)
    del(f_l)

    #BUILD NEW TRAIN FEATURE INPUT
    for idx_pic, picture in enumerate(picture_train_features):
        idx = idx_pic + (i*5000)
        x_train_feature_map[idx] = picture[0][0]

###### TEST data
f_t = np.empty([3589, 200, 200, 3])
for index, item in enumerate(f_t):  # Refill the list
    for d in range(3):
        item[0:48, 0:48,d] = test_data_X[index]


picture_test_features = resnetPreTrained.predict(f_t)
del(f_t)

#BUILD NEW TEST
x_test_feature_map  = np.empty([3589, 2048])
for idx_pic, picture in enumerate(picture_test_features):
    x_test_feature_map[idx_pic] = picture[0][0]
    
##### Validation data   
f_t = np.empty([3589, 200, 200, 3])
for index, item in enumerate(f_t):  # Refill the list
    for d in range(3):
        item[0:48, 0:48,d] = validation_data_X[index]


picture_validation_features = resnetPreTrained.predict(f_t)
del(f_t)

#BUILD NEW TEST
x_validation_feature_map  = np.empty([3589, 2048])
for idx_pic, picture in enumerate(picture_validation_features):
    x_validation_feature_map[idx_pic] = picture[0][0]


print("here")
with tf.device('/gpu:0'):
  model = Sequential()
  model.add(Dense(1024, input_shape=(2048,),activation='relu'))
  model.add(Dense(512, input_shape=(1024,),activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))
  adamax = Adamax()
  model.compile(loss='categorical_crossentropy', optimizer=adamax, metrics=['accuracy'])
  model.fit(x_train_feature_map, train_data_Y[0:N_TEST], validation_data=(x_validation_feature_map, validation_data_Y), epochs=10, batch_size=32)

  score = model.evaluate(x_test_feature_map, test_data_Y, batch_size=32)
  print("TeResult")
  print(score)


Train on 25000 samples, validate on 3589 samples  

Epoch 1/10
25000/25000 [==============================] - 8s 320us/step - loss: 2.5333 - acc: 0.2440 - val_loss: 1.8166 - val_acc: 0.2449  
Epoch 2/10
25000/25000 [==============================] - 6s 251us/step - loss: 2.5313 - acc: 0.2468 - val_loss: 1.8151 - val_acc: 0.2449  
Epoch 3/10
25000/25000 [==============================] - 6s 250us/step - loss: 2.5302 - acc: 0.2469 - val_loss: 1.8237 - val_acc: 0.2441      
Epoch 4/10
25000/25000 [==============================] - 6s 250us/step - loss: 2.5300 - acc: 0.2466 - val_loss: 1.8166 - val_acc: 0.2460     
Epoch 5/10
25000/25000 [==============================] - 6s 252us/step - loss: 2.5299 - acc: 0.2468 - val_loss: 1.8132 - val_acc: 0.2455   
Epoch 6/10
25000/25000 [==============================] - 6s 251us/step - loss: 2.5294 - acc: 0.2470 - val_loss: 1.8088 - val_acc: 0.2455   
Epoch 7/10
25000/25000 [==============================] - 6s 251us/step - loss: 2.5292 - acc: 0.2471 - val_loss: 1.8151 - val_acc: 0.2466   
Epoch 8/10
25000/25000 [==============================] - 6s 251us/step - loss: 2.5294 - acc: 0.2470 - val_loss: 1.8080 - val_acc: 0.2458   
Epoch 9/10
25000/25000 [==============================] - 6s 253us/step - loss: 2.5291 - acc: 0.2469 - val_loss: 1.8081 - val_acc: 0.2460   
Epoch 10/10
25000/25000 [==============================] - 6s 251us/step - loss: 2.5289 - acc: 0.2472 - val_loss: 1.8079 - val_acc: 0.2466  
3589/3589 [==============================] - 0s 78us/step  
Test Result
[loss = 1.8071214112773117,  Accuracy = 0.2471440512719145]

In [0]:
!ls
def load_image(img_path, show=True):

    img = image.load_img(img_path, target_size=(48, 48), color_mode = "grayscale")
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.
    #print(img_tensor.shape)
    if show:
        plt.imshow(img, cmap='gray')

    return img_tensor
    # show the image


if __name__ == "__main__":

    # load model
    #model = load_model("model.h5")

    # image path
    #!wget https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTyndHmil5Mdk8-v-6To_6ncm-l8-L5n0l0Bh5dcOR6ZUkq-pODmQ #Angry
    #!wget https://www.sciencedaily.com/images/2009/03/090312093916_1_540x360.jpg #Disgust
    #!wget https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcREqmMQoG0_0XOYf5YWhLIdo-xZmohGMOWm0P4aFUHpFzYiB_yY #Fear
    #!wget https://w3.chabad.org/media/images/689/tyZo6894268.jpg #Happy
    #!wget https://c1.staticflickr.com/4/3675/9436653177_fd00cc9d2c_b.jpg #Sad
    #!wget https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTt54l_3Sud9ektJH69BXBOADn3qk7d0XrTpjrAWJ6u5omaX2NlcQ # Surprise 
    #!wget https://goldenmeancalipers.com/wp-content/uploads/2011/12/mirror11.jpg # Netural

    # load a single image
    new_image = load_image('mirror11.jpg')
    
    # check prediction
    pred = model.predict(new_image)
    index = pred.argmax()
    print(result[index])
    #print(max(pred))
